In [1]:
# User configurations
#geo_json_file = 'data-1.json'
#geom_column = 'LGA_MAIN11'
#data_column = 'Tot_1941_1950'
#src_geom = 'LGA_2011'
#dst_geom = 'LGA_2016'

# Please select a GeoJSON file from the JupyteLab workspace.
geo_json_file = 'lga_abs_b23b.json'
# Please select from the input file the geometry column.
geom_column = 'LGA_MAIN11'
# Please select from the data file the data column.
data_column ='P_Vis_within_Aus_65_74'
# Please select the 'from' geometry from a list of available conversions.
src_geom = 'LGA_2011'
# Please select the 'to' geometry from a list of available conversions.
dst_geom = 'LGA_2016'

In [2]:
BASE_URL = 'https://ydyr.info/api/v1'

In [3]:
AUTH = ('<YDYR username>', '<YDYR password>')

In [4]:
# GeoJSON files directory
GEO_JSON_PATH = 'data'

In [5]:
# Downloaded result file directory
OUTPUT_PATH = 'results'

In [6]:
import json
import time
import requests

In [7]:
# post geo json data:
input_file = '/'.join((GEO_JSON_PATH, geo_json_file))
with open(input_file, 'r') as d:
    post_data = json.load(d)

In [8]:
def get_side_data(geom_column):
    year = int(geom_column.split('_')[1])
    side_data_year = 2011 if (abs(year - 2016) >= abs(year-2011)) else 2016
    return "BCP_" + str(side_data_year)

In [9]:
# post parameters:
post_parms = {
    'geom_column': geom_column,
    'data_column': data_column,
    'src_geom': src_geom,
    'dst_geom': dst_geom,
    'side_data': get_side_data(src_geom),
    'averaged_counts': False # XXX TODO
}

In [10]:
def get_disaggration(data, parms):
    res = requests.post(
        '/'.join((BASE_URL, 'disaggregate.geojson')),
        auth=AUTH,
        params=parms,
        json=data)
    assert res.status_code == 202, 'job failed to queue'

    job_id = res.json()
    print("Your submitted job id is", job_id)
    print()
    
    while res.status_code == 202:
        time.sleep(2)
        res = requests.get(
            '/'.join((BASE_URL, 'status', job_id)),
            auth=AUTH)
    assert res.status_code == 200, 'job failed to finish'

    res = requests.get(
        '/'.join((BASE_URL, 'download', res.json()['key'])),
        auth=AUTH,
        params={'format': 'json'})
    assert res.status_code == 200, 'failed to download result'

    #print("returned data:", res.json())
    
    output_file = '/'.join((OUTPUT_PATH, job_id+'.json'))
    with open(output_file, 'w') as f:
        json.dump(res.json(), f)
    print("Done! Your job result is downloaded at", output_file)
    print()

In [11]:
get_disaggration(post_data, post_parms)

Your submitted job id is e5eca8fa

Done! Your job result is downloaded at results/e5eca8fa.json

